## Import

In [158]:
# 데이터 분석
import pandas as pd
import numpy as np

In [159]:
# 시각화
import seaborn as sns
import matplotlib.pyplot as plt
import koreanize_matplotlib

In [160]:
# 정규표현식
import re

In [161]:
# 형태소분석 okt
from konlpy.tag import Okt

## 감성사전으로 라벨링 작업

### 어간 추출까지 완료된 파일불러오기

In [162]:
df_review_data = pd.read_csv('data/review_okt_final.csv', encoding='utf-8')

In [163]:
df_review_data

,review,시설분류,review2,review_okt,okt_stemmed
0,?,문화,NaN,NaN,NaN
1,친절해요,문화,친절해요,친절해요,친절하다
2,좋아요 만족스러워요,문화,좋아요 만족스러워요,좋아요 만족스러워요,좋다 만족스럽다
3,와우~~,문화,와우,와우,와우
4,굿,문화,굿,굿,굿
...,...,...,...,...,...
141240,설사를해서 병원갔어요 배를 만져보고 꼼꼼히 살펴보시더니 약은 안먹어도 되겠다고 하셔...,동물병원,설사를해서 병원갔어요 배를 만져보고 꼼꼼히 살펴보시더니 약은 안먹어도 되겠다고 하셔...,설사 해서 병원 갔어요 배 만져 보고 꼼꼼히 살펴보시더니 약 안 먹어도 되겠다고 하...,설사 하다 병원 가다 배 만지다 보고 꼼꼼하다 살펴보다 약 안 먹다 되다 하다 주사...
141241,저희 천사 데려온 곳이고 중성화부터 혈액검사까지 전부 세세하게 봐주셔서 늘 감사드립...,동물병원,저희 천사 데려온 곳이고 중성화부터 혈액검사까지 전부 세세하게 봐주셔서 늘 감사드립니다,저희 천사 데려온 곳 중성 화 혈액 검사 전부 세세하게 봐주셔서 늘 감사 드립니다,저희 천사 데려오다 곳 중성 화 혈액 검사 전부 세다 보다 늘 감사 드리다
141242,친절하십니다,동물병원,친절하십니다,친절하십니다,친절하다
141243,진료 꼼꼼히 잘봐주세요 진료후 간식도 챙겨주시고 선생님께서 산책 가자고하니 요미가 ...,동물병원,진료 꼼꼼히 잘봐주세요 진료후 간식도 챙겨주시고 선생님께서 산책 가자고하니 요미가 ...,진료 꼼꼼히 잘 봐주세요 진료 후 간식 챙겨주시고 선생님 산책 가자고하니 요미 따라...,진료 꼼꼼하다 자다 보다 진료 후 간식 챙기다 선생님 산책 가다 요미 따라가다 처방...


### 감성사전 불러오기

In [164]:
import json

In [165]:
with open('data/SentiWord_info.json', encoding='utf-8-sig', mode='r') as f:
    Sentiword_info = json.load(f)

In [166]:
sentiword_dic = pd.DataFrame(Sentiword_info)

In [167]:
sentiword_dic

,word,word_root,polarity
0,(-;,(,1
1,(;_;),(;_;),-1
2,(^^),(^^),1
3,(^-^),(^-^),1
4,(^^*,(,1
...,...,...,...
14849,반신반의하다,반신반의,0
14850,신비롭다,신비,1
14851,아리송하다,아리송,-1
14852,알쏭하다,알쏭하,-1


### 감성사전 추가하기

In [168]:
# 추가할 단어 있으면 추가하기
new_row = {'word': '친절해요', 'word_root': '친절', 'polarity' : '2'}
sentiword_dic = pd.concat([sentiword_dic, pd.DataFrame(new_row, index=[0])], ignore_index=True)

In [169]:
# 추가할 단어 있으면 추가하기
new_row = {'word': '친절하다', 'word_root': '친절하다', 'polarity' : '2'}
sentiword_dic = pd.concat([sentiword_dic, pd.DataFrame(new_row, index=[0])], ignore_index=True)

In [170]:
# 추가할 단어 있으면 추가하기
new_row = {'word': '굿', 'word_root': '굿', 'polarity' : '1'}
sentiword_dic = pd.concat([sentiword_dic, pd.DataFrame(new_row, index=[0])], ignore_index=True)

In [171]:
# 추가할 단어 있으면 추가하기
new_row = {'word': '와우', 'word_root': '와우', 'polarity' : '1'}
sentiword_dic = pd.concat([sentiword_dic, pd.DataFrame(new_row, index=[0])], ignore_index=True)

In [172]:
# 추가할 단어 있으면 추가하기
new_row = {'word': 'ㅋㅋ', 'word_root': 'ㅋㅋ', 'polarity' : '1'}
sentiword_dic = pd.concat([sentiword_dic, pd.DataFrame(new_row, index=[0])], ignore_index=True)

In [173]:
# 추가할 단어 있으면 추가하기
new_row = {'word': '덥다', 'word_root': '덥다', 'polarity' : '-1'}
sentiword_dic = pd.concat([sentiword_dic, pd.DataFrame(new_row, index=[0])], ignore_index=True)

In [174]:
# 추가할 단어 있으면 추가하기
new_row = {'word': '춥다', 'word_root': '춥다', 'polarity' : '-1'}
sentiword_dic = pd.concat([sentiword_dic, pd.DataFrame(new_row, index=[0])], ignore_index=True)

In [175]:
# 추가할 단어 있으면 추가하기
new_row = {'word': '좀더', 'word_root': '좀더', 'polarity' : '-1'}
sentiword_dic = pd.concat([sentiword_dic, pd.DataFrame(new_row, index=[0])], ignore_index=True)

In [176]:
# 추가할 단어 있으면 추가하기
new_row = {'word': '춥', 'word_root': '춥', 'polarity' : '-1'}
sentiword_dic = pd.concat([sentiword_dic, pd.DataFrame(new_row, index=[0])], ignore_index=True)

In [177]:
# 추가할 단어 있으면 추가하기
new_row = {'word': '힐링', 'word_root': '힐링', 'polarity' : '1'}
sentiword_dic = pd.concat([sentiword_dic, pd.DataFrame(new_row, index=[0])], ignore_index=True)

In [178]:
# 추가할 단어 있으면 추가하기
new_row = {'word': '자랑', 'word_root': '자랑', 'polarity' : '1'}
sentiword_dic = pd.concat([sentiword_dic, pd.DataFrame(new_row, index=[0])], ignore_index=True)

In [179]:
sentiword_dic.tail()

,word,word_root,polarity
14860,춥다,춥다,-1
14861,좀더,좀더,-1
14862,춥,춥,-1
14863,힐링,힐링,1
14864,자랑,자랑,1


### 라벨링 작업

In [180]:
# 감정사전 key-value(단어 - 긍정부정값)
hash_map = {}

row에 word_root를 넣으면 어간 word를 넣으면 단어

In [181]:
# 감정사전 key-value 값 해싱
for index, row in sentiword_dic.iterrows():
  hash_map[row['word']] = row['polarity']

In [182]:
# 사전 점수 합계 구할꺼
def calculate_sentiment_score(text):
    if pd.isna(text):
        return 0
    score = 0
    words = text.split()
    for word in words:
      score += int(hash_map.get(word,0))
    return score

In [183]:
# 사전 점수 합계 구해서 label
def calculate_sentiment_korean(text):
    if pd.isna(text):
        return 0
    score = 0
    words = text.split()
    for word in words:
      score += int(hash_map.get(word,0))

    if score > 0: return 1
    elif  score < 0 : return -1
    else : return 0

In [184]:
calculate_sentiment_score('헐')

0

In [186]:
# 리뷰 데이터에 대한 감정 라벨링
from tqdm import tqdm
tqdm.pandas()
df_review_data['score'] = df_review_data['okt_stemmed'].apply(calculate_sentiment_score)

In [187]:
# 리뷰 데이터에 대한 감정 라벨링
from tqdm import tqdm
tqdm.pandas()
df_review_data['label'] = df_review_data['okt_stemmed'].apply(calculate_sentiment_korean)

In [188]:
df_review_data

,review,시설분류,review2,review_okt,okt_stemmed,score,label
0,?,문화,NaN,NaN,NaN,0,0
1,친절해요,문화,친절해요,친절해요,친절하다,2,1
2,좋아요 만족스러워요,문화,좋아요 만족스러워요,좋아요 만족스러워요,좋다 만족스럽다,4,1
3,와우~~,문화,와우,와우,와우,1,1
4,굿,문화,굿,굿,굿,1,1
...,...,...,...,...,...,...,...
141240,설사를해서 병원갔어요 배를 만져보고 꼼꼼히 살펴보시더니 약은 안먹어도 되겠다고 하셔...,동물병원,설사를해서 병원갔어요 배를 만져보고 꼼꼼히 살펴보시더니 약은 안먹어도 되겠다고 하셔...,설사 해서 병원 갔어요 배 만져 보고 꼼꼼히 살펴보시더니 약 안 먹어도 되겠다고 하...,설사 하다 병원 가다 배 만지다 보고 꼼꼼하다 살펴보다 약 안 먹다 되다 하다 주사...,1,1
141241,저희 천사 데려온 곳이고 중성화부터 혈액검사까지 전부 세세하게 봐주셔서 늘 감사드립...,동물병원,저희 천사 데려온 곳이고 중성화부터 혈액검사까지 전부 세세하게 봐주셔서 늘 감사드립니다,저희 천사 데려온 곳 중성 화 혈액 검사 전부 세세하게 봐주셔서 늘 감사 드립니다,저희 천사 데려오다 곳 중성 화 혈액 검사 전부 세다 보다 늘 감사 드리다,1,1
141242,친절하십니다,동물병원,친절하십니다,친절하십니다,친절하다,2,1
141243,진료 꼼꼼히 잘봐주세요 진료후 간식도 챙겨주시고 선생님께서 산책 가자고하니 요미가 ...,동물병원,진료 꼼꼼히 잘봐주세요 진료후 간식도 챙겨주시고 선생님께서 산책 가자고하니 요미가 ...,진료 꼼꼼히 잘 봐주세요 진료 후 간식 챙겨주시고 선생님 산책 가자고하니 요미 따라...,진료 꼼꼼하다 자다 보다 진료 후 간식 챙기다 선생님 산책 가다 요미 따라가다 처방...,2,1


### 저장

In [101]:
df_review_data.to_csv('data/knu_review_okt.csv', index=False, encoding='utf-8-sig')

In [102]:
df_review_data.to_excel('data/knu_review_okt.xlsx', index=False)

분류

In [189]:
df_review_data[df_review_data['label'] == 0].shape

(22660, 7)

In [190]:
df_review_data[df_review_data['label'] == 1].shape

(112536, 7)

In [191]:
df_review_data[df_review_data['label'] == -1].shape

(6049, 7)

In [124]:
df_review_data[df_review_data['label'] == 0].to_csv('data/0인거.csv', index=False, encoding='utf-8-sig')

In [125]:
df_review_data[df_review_data['label'] == 1].to_csv('data/긍정인거.csv', index=False, encoding='utf-8-sig')

In [126]:
df_review_data[df_review_data['label'] == -1].to_csv('data/부정인거.csv', index=False, encoding='utf-8-sig')